# 求得每个种类每日的出售量

In [1]:
import pandas as pd
import numpy as np

In [2]:
data1 = pd.read_excel("附件1.xlsx")
kind = list(data1['分类编码'])
item = list(data1['单品编码'])
data1_dict = {}
for i in range(0,len(item)):
    data1_dict.setdefault(item[i],[]).append(kind[i])

In [3]:
kind_norepeat = [] # 对种类进行去重
repeat_times = {} # 记录每个种类各有多少品类
for i in kind:
    try:
        x = kind_norepeat.index(i)
        repeat_times[i] = repeat_times[i] + 1
        continue
    except Exception:
        kind_norepeat.append(i)
        repeat_times[i] = 1


In [4]:
data_eachkind = pd.read_excel("每日各种单品数据.xlsx",index_col=0,header=[0,1])

In [5]:
data_new = {}
data_new = pd.DataFrame(data_new, index=pd.to_datetime(data_eachkind.index).date,columns=kind_norepeat)
data_new = data_new.astype(np.float64,errors='ignore')
for i in range(0,len(data_new.index)):
    if i % 100 == 0:
        print(i)
    for j in range(0,len(item)):
        new_index = data_new.index[i]
        new_column = data1_dict[int(data_eachkind.columns[4*j][0])][0]
        
        read_data = data_eachkind.iloc[i,4*j]
        exist_data = data_new.loc[new_index,new_column]
        
        if np.isnan(read_data):
            continue
        else:
            if np.isnan(exist_data):
                data_new.loc[new_index,new_column] = read_data
            else:
                data_new.loc[new_index,new_column] = exist_data + read_data
print('Done')

0
100
200
300
400
500
600
700
800
900
1000
Done


In [6]:
data_new

,1011010101,1011010201,1011010402,1011010501,1011010504,1011010801
2020-07-01,205.402,46.640,4.850,35.374,76.715,35.365
2020-07-02,198.362,43.943,4.600,32.199,66.064,48.510
2020-07-03,190.779,42.076,9.572,35.896,64.253,42.442
2020-07-04,236.587,55.662,5.439,57.067,81.282,47.262
2020-07-05,223.899,55.474,4.019,61.816,98.496,73.213
...,...,...,...,...,...,...
2023-06-26,80.524,8.083,10.384,21.224,72.185,39.582
2023-06-27,127.278,14.272,16.454,8.415,67.120,38.708
2023-06-28,130.182,16.069,14.946,15.651,68.534,53.742
2023-06-29,135.090,24.367,22.945,11.511,89.113,48.314


# 由于每个种类所含品类不同，故考虑对每个种类取平均和平均一天售出量

In [7]:
data_origin = data_new

In [8]:
data_new = {}
data_new = pd.DataFrame(data_new,index=pd.to_datetime(data_origin.index).date,columns=data_origin.columns)
data_new = data_new.astype(np.float64,errors='ignore')

In [9]:
for i in range(0,len(data_new.columns)):
    column = data_new.columns[i]
    data_new[column] = (data_origin[column] / repeat_times[column]).round(3)
    data_new.loc['Average',column] = (data_new[column].sum() / len(data_new.index)).round(3)

In [10]:
with pd.ExcelWriter('每日各种类型出售量.xlsx') as writer:
    data_origin.to_excel(writer,sheet_name='Sheet1')
    data_new.to_excel(writer,sheet_name='Sheet2')

In [11]:
data_new

,1011010101,1011010201,1011010402,1011010501,1011010504,1011010801
2020-07-01,2.054,9.328,0.255,3.537,1.705,0.491
2020-07-02,1.984,8.789,0.242,3.220,1.468,0.674
2020-07-03,1.908,8.415,0.504,3.590,1.428,0.589
2020-07-04,2.366,11.132,0.286,5.707,1.806,0.656
2020-07-05,2.239,11.095,0.212,6.182,2.189,1.017
...,...,...,...,...,...,...
2023-06-27,1.273,2.854,0.866,0.842,1.492,0.538
2023-06-28,1.302,3.214,0.787,1.565,1.523,0.746
2023-06-29,1.351,4.873,1.208,1.151,1.980,0.671
2023-06-30,0.255,1.462,0.098,0.333,0.416,0.040
